# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag, ne_chunk
from nltk.corpus import wordnet

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download()

In [27]:
# load data from database
DB_NAME = '../data/DisasterResponse.db'
MESSAGES_TABLE = 'messages'

engine = create_engine('sqlite:///{}'.format(DB_NAME))
df = pd.read_sql_table(MESSAGES_TABLE, engine)
print(df.shape)
df.head()

(26216, 40)


,id,message,original,genre,aid_centers,aid_related,buildings,child_alone,clothing,cold,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
aid_centers                   0
aid_related                   0
buildings                     0
child_alone                   0
clothing                      0
cold                          0
death                         0
direct_report                 0
earthquake                    0
electricity                   0
fire                          0
floods                        0
food                          0
hospitals                     0
infrastructure_related        0
medical_help                  0
medical_products              0
military                      0
missing_people                0
money                         0
offer                         0
other_aid                     0
other_infrastructure          0
other_weather                 0
refugees                      0
related                       0
request                       0
search_a

In [29]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [30]:
# Let's keep only the English version of the messages, as the final classifier
# is based only in English messages
X = df.loc[:, 'message']
y = df.iloc[:, 4:]

print(X.shape)
print(y.shape)

(26216,)
(26216, 36)


In [31]:
category_names = y.columns.tolist()
print(category_names)

['aid_centers', 'aid_related', 'buildings', 'child_alone', 'clothing', 'cold', 'death', 'direct_report', 'earthquake', 'electricity', 'fire', 'floods', 'food', 'hospitals', 'infrastructure_related', 'medical_help', 'medical_products', 'military', 'missing_people', 'money', 'offer', 'other_aid', 'other_infrastructure', 'other_weather', 'refugees', 'related', 'request', 'search_and_rescue', 'security', 'shelter', 'shops', 'storm', 'tools', 'transport', 'water', 'weather_related']


In [32]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [33]:
y.head()

,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [34]:
text = X[3]
text

'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.'

In [35]:
def get_wordnet_pos(treebank_tag):
    """ 
    Transforms from Treebank tags to wordnet tags.
    As discussed here: 
    https://stackoverflow.com/questions/15586721/
    wordnet-lemmatization-and-pos-tagging-in-python
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # If unknown, return the default value

In [36]:
# Case normalization
temp_text = text.lower()

# Punctuation removal
temp_text = re.sub(r'[^a-zA-Z0-9]', ' ', temp_text)

# Tokenize
tokens = word_tokenize(temp_text)

# Stop Word Removal
stop_words = stopwords.words("english")
tokens = [word for word in tokens if word not in stop_words]

# Part-of-Speech Tagging
tokens = [(token[0], get_wordnet_pos(token[1])) for token in pos_tag(tokens)]

# Named Entity Recognition
# TODO: Add this to the pipeline. The punctuation is important to recognize the
# entities.

# Lemmatization
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(*token) for token in tokens]

# Stemming
stemmer = PorterStemmer()
tokens = [stemmer.stem(word) for word in tokens]

In [37]:
tokens

['un',
 'report',
 'leogan',
 '80',
 '90',
 'destroy',
 'hospit',
 'st',
 'croix',
 'function',
 'need',
 'suppli',
 'desper']

In [38]:
def tokenize(text):
    """ Basic tokenization function. """
    # Case normalization
    temp_text = text.lower()

    # Punctuation removal
    temp_text = re.sub(r'[^a-zA-Z0-9]', ' ', temp_text)

    # Tokenize
    tokens = word_tokenize(temp_text)

    # Stop Word Removal
    stop_words = stopwords.words("english")
    tokens = [word for word in tokens if word not in stop_words]

    # Part-of-Speech Tagging
    tokens = [(token[0], get_wordnet_pos(token[1])) for token in pos_tag(tokens)]

    # Named Entity Recognition
    # TODO: Add this to the pipeline. The punctuation is important to recognize the
    # entities.

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(*token) for token in tokens]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    
    return tokens

In [39]:
%time tokenize(X[0])

CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 10.4 ms


['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

In [40]:
def time_tokenize(X):
    result = list()
    for text in X[:10]:
        result.append(tokenize(text))
    return result

In [15]:
%timeit time_tokenize(X)

16.6 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
'Estimated tokenization time: {} seconds'.format((14.6 * 26200 / 10) / 1000)

'Estimated tokenization time: 38.252 seconds'

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [18]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x11293c620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, ve

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)
%time pipeline.fit(X_train, y_train)

CPU times: user 1min 31s, sys: 7.58 s, total: 1min 38s
Wall time: 1min 53s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [20]:
# What if I use all the cores for the classification part?
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])
%time pipeline.fit(X_train, y_train)

CPU times: user 1min 41s, sys: 3.11 s, total: 1min 44s
Wall time: 1min 7s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

A small improvement (the preprocessing part takes a lot of time).

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
%time y_pred = pipeline.predict(X_test)

CPU times: user 23.2 s, sys: 1.04 s, total: 24.2 s
Wall time: 23.4 s


In [22]:
y_pred.shape

(7865, 36)

In [23]:
for i in range(y_pred.shape[1]):
    print(classification_report(y_test.values[:, i], y_pred[:, i]))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      7769
          1       0.00      0.00      0.00        96

avg / total       0.98      0.99      0.98      7865

             precision    recall  f1-score   support

          0       0.74      0.86      0.80      4539
          1       0.75      0.59      0.66      3326

avg / total       0.75      0.75      0.74      7865

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      7439
          1       0.80      0.10      0.18       426

avg / total       0.94      0.95      0.93      7865

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7865

avg / total       1.00      1.00      1.00      7865

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      7742
          1       0.79      0.12      0.21       123

avg / total       0.98      0.99

/Users/antelinvestigacionydesarrollo/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Let's try to get a global score

In [24]:
y_pred.reshape(1, -1).shape

(1, 283140)

In [25]:
y_test.values.reshape(1, -1).shape

(1, 283140)

In [26]:
y_test.values.reshape(1, -1)[0].shape

(283140,)

In [27]:
from sklearn.metrics import f1_score

f1_score(y_test.values.reshape(1, -1)[0], y_pred.reshape(1, -1)[0], average='micro')

0.9432083068446705

What if the labels are takens as vectors?

In [28]:
(y_test.values == y_pred).all(axis=1).mean()

0.23763509218054674

### 6. Improve your model
Use grid search to find better parameters. 

In [29]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x11293c620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
               oob_score=False, random_state=None, v

In [30]:
parameters = {
    # 'features__text_pipeline__vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__smooth_idf': [True, False],
    'clf__estimator__max_depth': [None, 7]
    # 'clf__estimator__n_estimators': [10, 100],
}

cv = GridSearchCV(pipeline, parameters, n_jobs=-1)

In [31]:
cv.get_params()

{'cv': None,
 'error_score': 'raise',
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x11293c620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_j

In [32]:
%time cv.fit(X_train, y_train)

CPU times: user 1min 39s, sys: 3.13 s, total: 1min 42s
Wall time: 10min 3s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__smooth_idf': [True, False], 'clf__estimator__max_depth': [None, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
best_estimator = cv.best_estimator_
best_estimator.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x11293c620>, vocabulary=None)),
  ('tfidf', TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
            use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
               oob_score=False, random_sta

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [57]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def test_model(pipeline, X_train, X_test, y_train, y_test):
    print('Fitting the best estimator')
    %time best_estimator.fit(X_train, y_train)
    print('Predicting')
    %time y_pred = best_estimator.predict(X_test)
    
    print('Evaluating the results')
    results = list()
    for i in range(y_pred.shape[1]):
        acc = accuracy_score(y_test.values[:, i], y_pred[:, i])
        prec = precision_score(y_test.values[:, i], y_pred[:, i], average='macro')
        rec = recall_score(y_test.values[:, i], y_pred[:, i], average='macro')
        f1 = f1_score(y_test.values[:, i], y_pred[:, i], average='macro')
        results.append({'accuracy': acc,
                        'precision': prec,
                        'recall': rec,
                        'f1': f1})
    results_df = pd.DataFrame(results, index=y_test.columns)
    print(results_df.describe())
    
    return results_df

In [58]:
test_model(pipeline, X_train, X_test, y_train, y_test)

Fitting the best estimator
CPU times: user 1min 41s, sys: 2.97 s, total: 1min 44s
Wall time: 1min 7s
Predicting
CPU times: user 22.9 s, sys: 954 ms, total: 23.9 s
Wall time: 23.4 s
Evaluating the results
        accuracy         f1  precision     recall
count  36.000000  36.000000  36.000000  36.000000
mean    0.942590   0.603828   0.754881   0.585122
std     0.057919   0.126357   0.159884   0.114577
min     0.743166   0.486061   0.490396   0.499741
25%     0.930102   0.505610   0.644550   0.502050
50%     0.956771   0.558880   0.812255   0.541397
75%     0.982422   0.703763   0.865203   0.654049
max     1.000000   1.000000   1.000000   1.000000


/Users/antelinvestigacionydesarrollo/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/antelinvestigacionydesarrollo/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy,f1,precision,recall
aid_centers,0.987667,0.496898,0.493896,0.499936
aid_related,0.743166,0.726104,0.745815,0.721843
buildings,0.950540,0.597450,0.852906,0.563344
child_alone,1.000000,1.000000,1.000000,1.000000
clothing,0.984997,0.574787,0.816394,0.544328
cold,0.981564,0.599162,0.808115,0.561378
death,0.957661,0.589846,0.891033,0.556777
direct_report,0.841577,0.681593,0.790500,0.650429
earthquake,0.961348,0.868861,0.933765,0.823753
electricity,0.978004,0.516536,0.822454,0.511299


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

I will choose the mean f1 score as the main metric to improve.

### 9. Export your model as a pickle file

In [55]:
from sklearn.externals import joblib

joblib.dump(best_estimator, 'estimator.pkl')

['estimator.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.